In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm

In [3]:
class HoverNet(nn.Module):
    def __init__(self):
        super(HoverNet, self).__init__()
        # Define your HoverNet architecture here
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.conv2(x)
        return x

In [12]:
class CustomDataset(Dataset):
    def __init__(self, image_folder, mask_folder, transform=None):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_folder))  # Ensure images and masks are properly matched
        self.mask_paths = sorted(os.listdir(mask_folder))

    def __len__(self):
        return len(self.image_paths)  # Return the length of the dataset

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_folder, self.image_paths[idx])
        mask_path = os.path.join(self.mask_folder, self.mask_paths[idx])

        image = Image.open(image_path).convert('RGB')  # Open image in RGB mode
        mask = Image.open(mask_path).convert('L')  # Open mask as a grayscale image

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


In [13]:
# Define transforms if needed
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images and masks to the same size
    transforms.ToTensor(),  # Convert images and masks to tensors
])


In [14]:

# Define dataset and dataloader
image_folder = '/content/drive/MyDrive/Hover/Images'
mask_folder = '/content/drive/MyDrive/Hover/Mask'
batch_size = 8
learning_rate = 0.0001  # Set your desired learning rate value

dataset = CustomDataset(image_folder=image_folder, mask_folder=mask_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [15]:
import torch.nn as nn
import torch.optim as optim


In [16]:
# Instantiate your HoverNet model
hovernet = HoverNet()

In [17]:
# Define the loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(hovernet.parameters(), lr=learning_rate)


In [18]:
num_epochs = 20  # Define the number of epochs for training
from PIL import Image

In [19]:
for epoch in range(num_epochs):
    hovernet.train()  # Set model to training mode
    for images, masks in tqdm(dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        optimizer.zero_grad()  # Zero the gradients
        outputs = hovernet(images)  # Forward pass
        loss = criterion(outputs, masks.float())  # Convert masks to float
         # Calculate the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

Epoch 20/20: 100%|██████████| 3/3 [00:01<00:00,  2.20it/s]
